In [ ]:
import cv2
import os
from pathlib import Path
import sys
sys.path.append('..')
from augmentations import CenterCrop

In [ ]:
CROP = 1024
orig_dir = Path('../data/train_orig')
crop_dir = Path('../data/train_{}'.format(CROP))

try:
    crop_dir.mkdir()
except FileExistsError:
    print('!!! Will overwrite files, is it OK?')

In [ ]:
for class_dir in orig_dir.iterdir():
    print(class_dir)
    o_paths = [path for path in class_dir.glob('*.jpg')]
    o_paths.extend([path for path in class_dir.glob('*.JPG')])
    d_paths = [crop_dir / class_dir.name / p.name for p in o_paths]
    
    (crop_dir / class_dir.name).mkdir(exist_ok=True)
    for o_path in o_paths:
        # TIF does not loose quality
        d_path = crop_dir / class_dir.name / (o_path.stem + '.tif')
        img = cv2.imread(str(o_path))
        sz = min(img.shape[0], CROP), min(img.shape[1], CROP)
        img = CenterCrop(sz)(img)
        cv2.imwrite(str(d_path), img)